# Learning

In [96]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [97]:
MODEL = 'ResNet50'
batch_size=8
train_data_path = './CargoContainerImages/train/'
num_classes = 2
max_epoches = 10

In [98]:
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode ='categorical')

Found 147 images belonging to 2 classes.


In [99]:
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input
    model = ResNet50(weights = None, classes=num_classes)
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import VGG16
    from tensorflow.keras.applications.vgg16 import preprocess_input
    model = VGG16(weights = None, classes = num_classes)
    output_node_name = 'predictions/Softmax'


In [100]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/10
19/19 [==============================] - 50s 3s/step - loss: 2.8184 - accuracy: 0.6327
Epoch 2/10
19/19 [==============================] - 46s 2s/step - loss: 1.2366 - accuracy: 0.7755
Epoch 3/10
19/19 [==============================] - 49s 3s/step - loss: 0.7062 - accuracy: 0.8503
Epoch 4/10
19/19 [==============================] - 46s 2s/step - loss: 0.3785 - accuracy: 0.9048
Epoch 5/10
19/19 [==============================] - 47s 2s/step - loss: 0.3720 - accuracy: 0.9456
Epoch 6/10
19/19 [==============================] - 46s 2s/step - loss: 0.1655 - accuracy: 0.9320
Epoch 7/10
19/19 [==============================] - 47s 2s/step - loss: 0.2049 - accuracy: 0.9388
Epoch 8/10
19/19 [==============================] - 47s 2s/step - loss: 0.1770 - accuracy: 0.9592
Epoch 9/10
19/19 [==============================] - 46s 2s/step - loss: 0.0712 - accuracy: 0.9864
Epoch 10/10
19/19 [==============================] - 47s 2s/step - loss: 0.0170 - accuracy: 0.9932


In [77]:
MODEL = 'VGG16'
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(train_generator, epochs = max_epoches)
model.save('saved_model/' + MODEL + 'saved.h5')

Epoch 1/10
19/19 [==============================] - 14s 761ms/step - loss: 0.8431 - accuracy: 0.8571
Epoch 2/10
19/19 [==============================] - 14s 763ms/step - loss: 0.6698 - accuracy: 0.7483
Epoch 3/10
19/19 [==============================] - 14s 761ms/step - loss: 0.3462 - accuracy: 0.8707
Epoch 4/10
19/19 [==============================] - 14s 763ms/step - loss: 0.3482 - accuracy: 0.8980
Epoch 5/10
19/19 [==============================] - 15s 763ms/step - loss: 0.0866 - accuracy: 0.9728
Epoch 6/10
19/19 [==============================] - 14s 761ms/step - loss: 0.0935 - accuracy: 0.9796
Epoch 7/10
19/19 [==============================] - 15s 766ms/step - loss: 0.0343 - accuracy: 0.9932
Epoch 8/10
19/19 [==============================] - 15s 774ms/step - loss: 0.0236 - accuracy: 0.9932
Epoch 9/10
19/19 [==============================] - 14s 763ms/step - loss: 0.0118 - accuracy: 0.9932
Epoch 10/10
19/19 [==============================] - 15s 764ms/step - loss: 0.0429 - accura

# Prediction

In [123]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
#import uff

In [124]:
MODEL = 'ResNet50'

In [125]:
if MODEL == 'ResNet50':
    output_node_name = 'fc1000/Softmax'
if MODEL == 'VGG16':
    output_node_name  = 'predictions/Softmax'
#sess = K.get_session()
#K.set_learning_phase(0)
#graph = tf.get_default_graph()
model = load_model('saved_model/' + MODEL + 'saved.h5')
model.compile(loss = 'categorical_crossenthropy', optimizer = 'adam', metrics= ['accuracy'])
#output_graph_def = tf.graph_util.convert_variables_to_constants(sess, graph.as_graph_def(), ['input_1', output_node_name])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [126]:
#uffmodel = uff.from_tensorflow(output_graph_def, [output_node_name], 
#                              output_filename = 'saved_model/' + MODEL + 'uff')

In [127]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
if MODEL == 'ResNet50':
    from tensorflow.keras.applications.resnet50 import decode_predictions
if MODEL == 'VGG16':
    from tensorflow.keras.applications.vgg16 import decode_predictions
def predict_all(folder):
    for filename in os.listdir(folder):
        print("Predicting...", filename)
        fullpath = os.path.join(folder, filename)  # 폴더명과 파일명을 결합
        #print("fullpath", fullpath)  
        try:
            # 테스트할 이미지 불러오기
            img = load_img(fullpath, target_size=(224, 224)) 
            # ResNet에 입력하기 전에 이미지 전처리
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            # 이미지 분류
            preds = model.predict(x)
            print('horizontal:vertical   ', preds)
            print('Predicted:', decode_predictions(preds, top=1)[0])

        except Exception as e:
            print(e)

In [116]:
predict_all('./CargoContainerImages/train/1/')

Predicting... BHCU2088072.jpg
horizontal:vertical    [[8.63184937e-07 1.95169491e-06 4.98553436e-06 6.84362305e-07
  5.89441470e-07 2.07900348e-06 1.51675607e-07 3.61387424e-06
  1.19987055e-06 8.76842535e-07 3.37137112e-06 6.94800121e-07
  2.02775846e-06 4.59600051e-06 2.56043177e-06 4.70807072e-06
  1.79522203e-06 1.53662550e-05 4.79417338e-07 3.77656670e-06
  7.10599465e-07 4.82885162e-06 6.96001791e-07 7.78767287e-07
  5.97817507e-06 3.79411290e-07 2.82360304e-07 1.93850497e-07
  1.69936595e-06 3.03155707e-06 7.24738172e-08 1.51435529e-07
  2.31168741e-07 3.28785859e-07 7.82514974e-07 1.54140125e-07
  3.82284583e-07 4.94066626e-07 7.70910162e-07 7.62958337e-07
  9.71191980e-07 1.54828285e-07 3.25176863e-08 1.37477642e-07
  3.45336076e-07 1.72411160e-06 6.44494662e-08 1.41618344e-07
  3.04637274e-06 1.38611995e-06 9.01036913e-07 7.79062975e-05
  1.86962424e-07 2.70867190e-07 2.86058679e-07 9.45026329e-07
  1.44779719e-07 3.17967839e-08 5.41408156e-07 1.28622025e-06
  4.56775723e-07 

horizontal:vertical    [[1.42678702e-10 3.00309638e-10 5.17410767e-11 1.60830273e-11
  2.18784036e-11 2.24400901e-10 1.24925905e-12 1.06862845e-10
  2.23921201e-10 3.76582904e-10 1.92200456e-09 1.01129979e-10
  8.54458448e-11 5.66306446e-10 4.03995795e-11 1.66439001e-10
  5.98005312e-10 4.52597321e-10 6.49901383e-11 1.05178455e-09
  4.46005385e-11 1.45900181e-10 3.31584690e-11 8.18546134e-11
  1.35975259e-10 1.84305609e-11 6.56798487e-12 2.44337172e-11
  2.41357941e-11 1.19235954e-10 1.43798229e-11 6.89086028e-12
  6.28928463e-12 3.47593699e-12 1.38411843e-11 3.32973102e-12
  1.05942624e-11 2.43428697e-11 6.38040246e-11 3.21063662e-11
  2.07332259e-11 2.67320974e-11 2.71493765e-12 7.05152820e-12
  1.16912478e-11 3.18180315e-10 3.19019182e-12 1.68372261e-11
  5.44358170e-10 1.58061342e-11 1.02181257e-10 4.17597867e-09
  4.16728561e-11 2.57214458e-11 1.18577786e-10 3.81763024e-11
  1.22583541e-11 3.14224276e-12 6.87424492e-11 5.13559577e-11
  3.96023873e-11 1.36800606e-11 2.39802483e-10 

horizontal:vertical    [[6.58311738e-10 4.36353315e-10 3.63916608e-09 5.21145731e-11
  3.30011296e-10 1.29096689e-08 2.71656697e-10 1.14481022e-10
  4.15104923e-10 2.78983420e-10 5.55872348e-10 1.51038598e-10
  3.91013090e-11 1.12999755e-10 9.93354357e-11 3.69011849e-10
  1.38693435e-09 1.39356013e-10 9.14259016e-11 1.87016913e-09
  2.84137037e-11 6.00576089e-10 2.88512630e-10 5.08400266e-10
  2.88174373e-10 6.23619212e-11 4.16479039e-11 1.60865640e-10
  1.98776343e-10 7.75368242e-11 1.67973569e-11 9.12029688e-11
  2.21419983e-11 2.66863510e-11 9.67731190e-12 8.64391631e-12
  1.17016494e-10 5.56049165e-11 2.89425345e-10 4.24886404e-10
  9.73084530e-11 2.72198680e-10 7.27942099e-12 8.92988322e-11
  1.19867285e-11 6.00434036e-10 8.44601559e-12 3.76585568e-11
  5.57753232e-10 9.69379507e-11 8.90044774e-11 2.84599196e-07
  1.03825622e-10 1.62815508e-11 9.24219035e-12 4.39660565e-11
  2.17192826e-11 1.16428698e-11 2.34856683e-11 1.27799313e-10
  6.31787886e-11 6.00964278e-11 3.35485950e-09 

horizontal:vertical    [[9.90758764e-08 1.69401645e-07 5.08519555e-08 6.30719921e-09
  1.54071031e-08 4.74153069e-07 2.67335754e-09 9.43264578e-09
  1.33135529e-07 1.64567822e-07 2.61560189e-07 2.64126271e-07
  1.58941248e-07 1.05762346e-07 1.22681556e-07 3.09979669e-08
  1.06784194e-06 1.19785369e-07 3.05229158e-08 6.95157610e-07
  6.89093369e-08 3.18864863e-07 1.04081606e-08 6.21675369e-08
  3.88052577e-08 1.34953373e-08 1.58287694e-08 1.26451178e-08
  4.69212402e-08 2.39146143e-08 1.69473626e-08 9.34861148e-08
  1.81622823e-08 2.01084536e-08 1.43778287e-08 2.88702395e-09
  8.80033024e-09 3.18536131e-08 8.85353515e-08 1.42245042e-08
  2.58548543e-08 8.59831104e-08 4.61475391e-09 1.64098211e-08
  5.66125813e-09 1.61542317e-07 7.37570582e-09 1.63391114e-08
  9.28416171e-08 2.16330509e-08 1.56321605e-08 6.26807378e-07
  6.55538841e-08 3.22267617e-08 1.13114051e-08 1.92157490e-08
  3.57874685e-09 3.08563775e-09 4.37203092e-08 1.93708473e-07
  3.45877886e-08 1.30916611e-08 1.22369386e-08 

  2.00540740e-06 1.36356775e-07 1.20480547e-06 3.16787846e-05]]
Predicted: [('n04590129', 'window_shade', 0.67850643)]
Predicting... CICU9679964.jpg
horizontal:vertical    [[1.81970256e-06 3.53983910e-06 3.57066574e-06 3.92123502e-06
  9.69491339e-07 5.58569518e-05 2.84175513e-07 1.14568945e-06
  1.25203087e-06 4.26767429e-06 2.03067952e-06 9.38841026e-07
  1.56758995e-06 1.58473620e-06 8.26179712e-06 5.57616772e-07
  5.74562682e-06 1.58075964e-05 1.20831555e-06 5.64536685e-06
  2.90692668e-07 7.17619378e-06 1.32138700e-06 8.86307646e-07
  3.06837865e-06 2.26711649e-07 1.45905958e-07 4.45570862e-07
  2.62064276e-07 1.84820897e-06 6.31251496e-08 1.72859274e-07
  9.32586772e-08 3.85416058e-07 3.70101930e-06 2.08452619e-07
  3.35355736e-07 2.04586371e-07 1.17786351e-06 8.83155508e-07
  1.69652864e-07 7.56619215e-07 1.17114460e-07 2.85225440e-07
  2.08459184e-07 2.49340633e-06 1.27112600e-07 1.88004876e-07
  8.67054666e-07 2.71627528e-07 5.70250450e-07 1.71693304e-04
  3.40445212e-07 1.921

...

horizontal:vertical    [[3.98051903e-09 4.12189438e-09 1.28357691e-09 6.46206477e-10
  2.44775977e-10 4.61574823e-09 7.03538660e-12 1.46772261e-10
  2.29051902e-10 1.50012980e-09 1.12804388e-09 6.49572058e-08
  2.99938563e-09 2.63243677e-10 2.06452757e-08 2.33447511e-10
  2.12902709e-08 3.39870398e-09 1.35806130e-10 9.13805742e-09
  7.84656923e-11 4.49454562e-09 6.06576456e-11 3.34002909e-10
  1.01937243e-10 6.57016247e-11 4.73728591e-11 6.58033281e-11
  8.36538408e-11 1.45358747e-09 1.95992098e-10 4.33177688e-10
  6.20118623e-10 1.58907665e-10 7.88509674e-11 1.94886381e-11
  9.84984039e-11 7.97052979e-11 5.98102845e-10 2.80527879e-10
  1.39208436e-10 5.97507654e-10 1.18050222e-11 2.57009330e-10
  1.08553444e-10 4.14943990e-09 8.89624693e-11 2.88445912e-09
  1.55338009e-09 3.11433018e-10 3.08942261e-10 3.98701019e-08
  7.56091578e-10 6.20603430e-11 3.97241684e-10 4.25955243e-11
  6.09382753e-11 2.26797730e-11 3.44611867e-10 6.63979371e-10
  2.96737634e-10 1.49325788e-10 1.91258254e-09 

horizontal:vertical    [[5.50801538e-09 3.01693071e-08 8.43129087e-08 3.95115940e-09
  1.57149866e-08 1.13267525e-07 2.05675876e-09 1.80205774e-07
  9.34361282e-08 5.00560589e-07 6.97336660e-08 3.11549428e-07
  1.29655689e-07 8.50419539e-08 1.08000151e-07 6.66509621e-08
  3.59758189e-07 5.98366285e-07 1.16850977e-07 5.48965602e-07
  3.58133398e-08 6.62556303e-08 4.71658712e-09 4.28560867e-08
  2.73932326e-08 6.39958131e-09 4.25685459e-10 1.68999870e-09
  7.49366524e-09 4.07280565e-09 3.18919358e-09 1.83829858e-08
  1.39898662e-08 1.98089243e-08 4.57531923e-09 4.77931417e-10
  1.75240700e-09 1.56693130e-08 9.82698634e-09 6.18338580e-09
  1.61276972e-07 9.46513268e-09 1.26071487e-09 6.14321660e-09
  1.40606427e-09 1.94130791e-08 7.02186576e-09 5.68995162e-09
  3.76357114e-08 3.38671882e-08 3.65823505e-09 6.89883564e-07
  1.76854886e-08 7.65888775e-09 6.59420563e-09 5.83776494e-09
  3.16135251e-09 1.78773729e-09 2.81408976e-08 7.52438734e-09
  4.00482634e-08 1.08655387e-08 4.80567941e-09 

horizontal:vertical    [[1.72840853e-09 4.51252458e-09 2.90638402e-09 9.24905208e-10
  2.38031012e-10 3.83258190e-08 5.25094482e-11 1.29973599e-09
  3.57793550e-09 6.18812246e-09 6.47605205e-08 2.07150563e-08
  5.29353716e-09 2.14768581e-09 6.09062800e-09 1.92992311e-09
  6.28937347e-09 5.92233240e-09 1.66819714e-09 8.99652903e-08
  3.22512655e-10 2.21888885e-09 1.42132917e-10 2.69212042e-10
  7.46518014e-10 5.56134860e-10 4.59204647e-10 4.45789211e-10
  2.52163024e-09 1.18905303e-08 2.39579662e-10 1.17795085e-09
  1.66012215e-09 1.47992096e-09 9.07367181e-10 2.99972214e-10
  5.29904298e-09 1.21017085e-09 1.16014409e-09 7.48689666e-10
  5.80270998e-10 1.14266974e-09 3.00087233e-10 6.62464028e-10
  1.03462838e-09 1.85674764e-08 5.25411069e-11 1.47762225e-09
  1.84424955e-08 8.58468630e-10 5.80003212e-10 5.30262184e-07
  2.66717515e-09 1.38969769e-09 6.75048295e-10 1.39577225e-10
  2.73176787e-10 1.42195283e-10 1.88859151e-10 3.65373376e-09
  5.62467239e-10 1.70825243e-09 8.49724380e-09 

horizontal:vertical    [[5.10546494e-09 9.78098669e-08 1.91675724e-08 1.56342725e-08
  9.20561671e-09 1.17543003e-07 1.02442865e-09 7.74154874e-09
  1.22734942e-08 4.72267971e-07 4.91673973e-08 2.19050605e-07
  2.32244570e-08 3.89838100e-08 1.49587294e-08 2.66311879e-08
  4.24253059e-08 3.24852074e-08 9.35390165e-09 8.60044622e-07
  2.73101275e-09 1.27786180e-07 3.67739839e-09 1.78424067e-08
  4.64127190e-08 6.01940320e-09 2.61374589e-09 1.16808585e-09
  9.01981489e-10 1.53794659e-07 3.05233017e-09 4.54389193e-09
  4.88502039e-09 7.00979896e-09 4.02309386e-09 1.15959498e-09
  1.02705480e-08 2.81597989e-09 1.36327616e-08 9.44543075e-08
  6.21283736e-09 5.40121867e-08 1.44471179e-09 4.80012368e-08
  2.08289430e-09 1.44312281e-07 1.07481524e-09 9.89145565e-09
  3.61109301e-07 3.97466344e-08 1.15543832e-08 5.66946164e-06
  9.48147161e-09 1.67022773e-09 3.99522104e-09 1.07442377e-09
  4.51837101e-09 4.10610407e-10 9.22111631e-09 1.85902884e-08
  2.22495373e-08 8.16386425e-09 2.60027093e-07 

horizontal:vertical    [[3.39608053e-09 2.79879959e-08 1.68840430e-07 9.63148672e-09
  4.39586056e-09 8.28107076e-08 1.54698435e-10 1.20200729e-08
  2.15113083e-08 1.04748850e-07 4.41747225e-08 5.10745650e-08
  3.61375960e-08 1.60500964e-08 3.31185497e-08 8.58582094e-09
  2.38514826e-08 3.84255152e-08 5.80075454e-09 2.50010402e-07
  1.86121096e-09 3.54448981e-09 7.20091708e-10 1.49561128e-08
  4.01774347e-09 2.55136356e-10 1.81309801e-10 3.38116146e-10
  1.66367564e-09 1.93042506e-08 6.80973777e-10 1.99538297e-09
  1.42591894e-09 3.97331412e-09 5.95335337e-09 2.57643074e-10
  4.53405091e-09 1.72436854e-09 1.73288806e-09 4.84954121e-09
  2.89050139e-09 9.11405618e-09 1.74170817e-10 3.76951448e-09
  1.46017076e-09 1.87938713e-08 8.03867473e-10 2.91165003e-09
  1.12243237e-08 7.95984523e-09 3.45749163e-09 7.10273116e-06
  1.31846767e-09 1.30482070e-09 3.29135563e-09 8.90445329e-10
  1.75431514e-09 9.16290877e-10 4.55473215e-09 4.61299487e-09
  9.60690105e-09 6.14679818e-09 3.05997538e-08 

In [112]:
predict_all('./CargoContainerImages/train/2/')

Predicting... BMOU4699433.jpg
horizontal:vertical    [[1. 0.]]
Predicting... BMOU5782049.jpg
horizontal:vertical    [[1. 0.]]
Predicting... BSIU2575064.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CAIU2728316.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CAIU4051570.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CAIU8121016.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CARU3858347.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CAXU8004650.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CBXU2785862.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CMAU5349828.jpg
horizontal:vertical    [[1. 0.]]
Predicting... crop002.jpg
horizontal:vertical    [[1. 0.]]
Predicting... CRXU1525331.jpg
horizontal:vertical    [[1. 0.]]
Predicting... DFSU7696722.jpg
horizontal:vertical    [[1. 0.]]
Predicting... DFSU7696722_2.jpg
horizontal:vertical    [[1. 0.]]
Predicting... DFSU7697586.jpg
horizontal:vertical    [[1. 0.]]
Predicting... DLSU5000210.JPG
horizontal:vertical    [[1.

In [71]:
a = 86
b=61
a/(a+b)

0.5850340136054422

# Extra

In [114]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
 
# imagenet에 미리 훈련된 ResNet50 모델 불러오기
model = ResNet50(weights='imagenet')

 
# 테스트할 이미지 불러오기
img_path = 'test.jpg'
img = load_img(img_path, target_size=(224, 224)) 
 
# ResNet에 입력하기 전에 이미지 전처리
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
 
# 이미지 분류
preds = model.predict(x)
print(preds)
print('Predicted:', decode_predictions(preds, top=3)[0])


[[3.71102676e-10 1.56775593e-09 6.33899333e-07 4.02285906e-07
  1.18995573e-07 3.43044704e-08 1.14061187e-08 1.86256255e-09
  1.97986072e-09 3.11168064e-10 1.82857895e-10 3.71855130e-10
  2.36878317e-09 3.67928488e-09 3.19695798e-10 8.83174811e-10
  3.13072803e-08 3.11225201e-09 3.42527173e-10 3.32528387e-08
  1.96200078e-09 5.57546912e-07 2.18191012e-08 1.51773691e-07
  3.04395904e-08 3.22587219e-08 9.52947854e-09 3.20642490e-09
  2.90862823e-09 5.38676925e-09 4.07433504e-10 1.42372265e-08
  8.15091727e-09 2.41507152e-08 4.86918026e-08 2.22863115e-08
  1.41772318e-08 1.19823573e-09 1.62482561e-09 4.28137570e-09
  4.24195301e-09 1.14656548e-10 2.69182348e-08 1.31458524e-08
  4.61131089e-09 1.36950917e-09 1.47408766e-10 1.41981502e-08
  8.25802760e-10 4.84810414e-09 3.96564914e-09 2.89273453e-06
  4.81442397e-09 1.40846157e-09 1.27366040e-09 4.13008294e-10
  1.33297648e-08 5.33506173e-09 2.46092835e-09 8.66849162e-11
  1.01277942e-09 4.18089989e-08 1.58589213e-08 1.74744330e-10
  6.8164